# XML形式

## データサンプル

## XML形式のデータを読み込む

In [1]:
import xml.etree.ElementTree as ET

tree = ET.parse("data/ueno-map.kml")
root = tree.getroot()

In [2]:
namespace = "{http://www.opengis.net/kml/2.2}"
name = root[0].find(f"{namespace}name").text
name

'上野駅周辺飲食店'

In [3]:
folders = root[0].findall(f"{namespace}Folder")  # Folderノードを取得  # ・・①
out = []
for i, folder in enumerate(folders):  # Folderノードを繰り返し処理
    folder_name = folder.find(
        f"{namespace}name"
    ).text  # Folderノードのnameがカテゴリー名  # ・・②

    shops = folder.findall(f"{namespace}Placemark")  # 店舗の情報にアクセス
    shops_list = []
    for shop in shops:  # ・・③
        shop_name = shop.find(f"{namespace}name").text  # 店舗の名称を取得
        coordinates = (
            shop.find(f"{namespace}Point")
            .find(f"{namespace}coordinates")
            .text  # ・・④
        )  # 経度、緯度、高度の情報を取得
        lon, lat, alt = [
            s.strip() for s in coordinates.split(",", 2)  # ・・⑤
        ]  # 情報を分割
        # 店舗の情報を辞書にしてリストに追記
        shops_list.append(  # ・・⑥
            {
                "name": shop_name,
                "lon": lon,
                "lat": lat,
                "alt": alt,
            }
        )

    # カテゴリーごとの情報を辞書型にして、リストに追記
    out.append(  # ・・⑦
        {
            "id": i + 1,
            "name": folder_name,
            "shops": shops_list,
        }
    )
out

[{'id': 1,
  'name': 'ラーメン屋',
  'shops': [{'name': '東京じゃんがら ecute上野店',
    'lon': '139.7771312',
    'lat': '35.7140434',
    'alt': '0'},
   {'name': '博多ラーメン 長浜や 上野店',
    'lon': '139.7775269',
    'lat': '35.7124554',
    'alt': '0'},
   {'name': "T'sたんたん エキュート上野店",
    'lon': '139.7771408',
    'lat': '35.7140681',
    'alt': '0'},
   {'name': '鯛塩そば灯花 アトレ上野店',
    'lon': '139.7770178',
    'lat': '35.7133078',
    'alt': '0'},
   {'name': '横浜家系ラーメン 壱角家 上野店',
    'lon': '139.7778759',
    'lat': '35.7123833',
    'alt': '0'},
   {'name': '麺屋 天王', 'lon': '139.7797678', 'lat': '35.7141222', 'alt': '0'},
   {'name': '一蘭 アトレ上野山下口店',
    'lon': '139.7752174',
    'lat': '35.7124196',
    'alt': '0'},
   {'name': 'らーめん花の華 上野店',
    'lon': '139.7788319',
    'lat': '35.7121706',
    'alt': '0'},
   {'name': '六厘舎 上野店', 'lon': '139.7750484', 'lat': '35.7121834', 'alt': '0'},
   {'name': '横浜家系ラーメン希家 上野店',
    'lon': '139.7786098',
    'lat': '35.7118269',
    'alt': '0'}]},
 {'id': 2,
  'name'

## pandasのオブジェクトに変換する

In [4]:
import pandas as pd

df = pd.json_normalize(
    out,
    record_path="shops",
    meta=["name"],
    record_prefix="shop.",
)

In [5]:
df.head()

,shop.name,shop.lon,shop.lat,shop.alt,name
0,東京じゃんがら ecute上野店,139.7771312,35.7140434,0,ラーメン屋
1,博多ラーメン 長浜や 上野店,139.7775269,35.7124554,0,ラーメン屋
2,T'sたんたん エキュート上野店,139.7771408,35.7140681,0,ラーメン屋
3,鯛塩そば灯花 アトレ上野店,139.7770178,35.7133078,0,ラーメン屋
4,横浜家系ラーメン 壱角家 上野店,139.7778759,35.7123833,0,ラーメン屋


In [6]:
df.tail()

,shop.name,shop.lon,shop.lat,shop.alt,name
25,焼肉ホルモン本舗 炎 上野駅前店,139.7762069,35.7112493,0,焼肉屋
26,肉の街 上野本店,139.775175,35.711006,0,焼肉屋
27,太昌園上野駅前店,139.7754445,35.7104037,0,焼肉屋
28,焼肉ひだり,139.7810698,35.711169,0,焼肉屋
29,土古里 上野バンブーガーデン店,139.7745426,35.7124233,0,焼肉屋


### Note: RSS

In [8]:
import feedparser

url = "https://blog.python.org/feeds/posts/default?alt=rss"
d = feedparser.parse(url)

In [ ]:
d["feed"]["title"]  # RSSのタイトルを取得

In [9]:
for entry in d.entries:
    print(entry.date, end=", ")  # エントリーの日付を出力
    print(entry.title)  # エントリーごとのタイトルを出力

2024-05-08T18:11:10.014-04:00, Python 3.13.0 beta 1 released
2024-04-09T15:16:28.923-04:00, Python 3.12.3 and 3.13.0a6 released
2024-04-08T04:50:48.835-04:00, Python 3.11.9 is now available
2024-03-19T20:39:05.768-04:00, Python 3.10.14, 3.9.19, and 3.8.19 is now available
2024-03-12T20:01:02.277-04:00, Python 3.13.0 alpha 5 is now available
2024-02-15T16:09:52.409-05:00, Python 3.13.0 alpha 4 is now available
2024-02-07T10:07:21.471-05:00, Python 3.12.2 and 3.11.8 are now available.
2024-01-17T19:30:36.556-05:00, Python 3.13.0 alpha 3 is now available.
2023-12-10T11:10:22.240-05:00, Python 3.11.7 is now available
2023-12-07T19:50:22.570-05:00, Python 3.12.1 is now available
2023-11-22T19:46:04.221-05:00, Python 3.13.0 alpha 2 is now available
2023-10-13T08:31:36.016-04:00, Python 3.13.0 alpha 1 is now available
2023-10-03T05:25:29.509-04:00, Python 3.11.6 is now available
2023-10-02T12:00:23.445-04:00, Python 3.12.0 (final) now available
2023-09-19T07:21:28.701-04:00, Python 3.12.0 rel